## Computer Vision 16th Assignment
## Melika Heydari Rad 
#### Maktabkhooneh Course 

___

### توضیح کد با استفاده از کامنت ها میان قسمت های مختلف 

In [ ]:
#!/usr/bin/env python

'''
Camshift ردیاب
================

این برنامه یک نسخه آزمایشی از ردیابی مبتنی بر 
Meanshift
است.
شما یک شی رنگی را انتخاب میکنید و این برنامه آن را دنبال میکند.
این برنامه از دوربین فریم ها را میخواند. به صورت پیش فرض وبکم سیستم را. و هر دوربینی که کاربر وارد کند.

[1] http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.14.7673

نحوه استفاده :
------
    camshift.py [<video source>]

    برای شروع ردیابی، محدوده شی را با موس انتخاب کنید.

کلید های مورد استفاده :
-----
    ESC   - خروج
    b     - back-projected probability تغییر وضعیت برای نمایش
'''

# Python 2/3 سازگاری با
from __future__ import print_function
import sys
PY3 = sys.version_info[0] == 3

if PY3:
    xrange = range

import numpy as np
import cv2 as cv

# ایمپورت ماژول های محلی
import video
from video import presets

# تعریف کلاس برای برنامه 
class App(object):
    def __init__(self, video_src):
        # video capture ساختن شی از کلاس
        self.cam = video.create_capture(video_src, presets['cube'])
        # خواندن اولین فریم
        _ret, self.frame = self.cam.read()
        # ساختن پنجره اسم گذاری شده برای برنامه 
        cv.namedWindow('camshift')
        # تنظیم ماوس کالبک برای پنجره اسم گذاری شده 
        cv.setMouseCallback('camshift', self.onmouse)

        # مقدار دهی اولیه به متغیر ها
        self.selection = None
        self.drag_start = None
        self.show_backproj = False
        self.track_window = None

    # mouse callback تعریف تابع برای
    def onmouse(self, event, x, y, flags, param):
        if event == cv.EVENT_LBUTTONDOWN:
            self.drag_start = (x, y)
            self.track_window = None
        if self.drag_start:
            xmin = min(x, self.drag_start[0])
            ymin = min(y, self.drag_start[1])
            xmax = max(x, self.drag_start[0])
            ymax = max(y, self.drag_start[1])
            self.selection = (xmin, ymin, xmax, ymax)
        if event == cv.EVENT_LBUTTONUP:
            self.drag_start = None
            self.track_window = (xmin, ymin, xmax - xmin, ymax - ymin)

    # تعریف تابع برای نمایش هیستوگرام
    def show_hist(self):
        bin_count = self.hist.shape[0]
        bin_w = 24
        img = np.zeros((256, bin_count*bin_w, 3), np.uint8)
        for i in xrange(bin_count):
            h = int(self.hist[i])
            cv.rectangle(img, (i*bin_w+2, 255), ((i+1)*bin_w-2, 255-h), (int(180.0*i/bin_count), 255, 255), -1)
        img = cv.cvtColor(img, cv.COLOR_HSV2BGR)
        cv.imshow('hist', img)

    # تعریف تابع برای اجرای برنامه 
    def run(self):
        while True:
            # خواندن فریم بعدی
            _ret, self.frame = self.cam.read()
            # ساختن یک کپی از این فریم برای نمایش
            vis = self.frame.copy()
            # HSV تبدیل به فضای رنگی 
            hsv = cv.cvtColor(self.frame, cv.COLOR_BGR2HSV)
            # تعریف ماسک باینری برای شی مشخص شده
            mask = cv.inRange(hsv, np.array((0., 60., 32.)), np.array((180., 255., 255.)))

            # اگر شی انتخاب شده، هیستوگرام آن را محاسبه کن و نمایش بده 
            if self.selection:
                x0, y0, x1, y1 = self.selection
                hsv_roi = hsv[y0:y1, x0:x1]
                mask_roi = mask[y0:y1, x0:x1]
                hist = cv.calcHist( [hsv_roi], [0], mask_roi, [16], [0, 180] )
                cv.normalize(hist, hist, 0, 255, cv.NORM_MINMAX)
                self.hist = hist.reshape(-1)
                self.show_hist()

                vis_roi = vis[y0:y1, x0:x1]
                cv.bitwise_not(vis_roi, vis_roi)
                vis[mask == 0] = 0

            # If a track window is defined, apply CamShift to the back projection of the histogram
            # اگر ترک-ویندو تعریف شد، کم-شیفت را روی بک-پراجکشن هیستوگرام آن اعمال کن 
            if self.track_window and self.track_window[2] > 0 and self.track_window[3] > 0:
                self.selection = None
                prob = cv.calcBackProject([hsv], [0], self.hist, [0, 180], 1)
                prob &= mask
                term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1 )
                track_box, self.track_window = cv.CamShift(prob, self.track_window, term_crit)

                # اگر بک-پراجکشن فعال است، آن را نمایش بده ، (با فشردن کلید بی این متغیر تغییر میکند) 
                if self.show_backproj:
                    vis[:] = prob[...,np.newaxis]
                try:
                    # دور شی را بیضی رسم کن 
                    cv.ellipse(vis, track_box, (0, 0, 255), 2)
                except:
                    print(track_box)

            # در پنجره اسم گذاری شده  نمایش بده
            cv.imshow('camshift', vis)

            # منتظر فشردن کلید بی یا اسکیپ بمان به مدت ۵ میلی ثانیه
            ch = cv.waitKey(5)
            if ch == 27:
                break
            if ch == ord('b'):
                self.show_backproj = not self.show_backproj
        # وقتی برنامه تمام شد همه پنجره ها را ببند
        cv.destroyAllWindows()


if __name__ == '__main__':
    print(__doc__)
    import sys
    try:
        video_src = sys.argv[1]
    except:
        video_src = 0
    # یک شی از برنامه بساز و آن را اجرا کن 
    App(video_src).run()

In [ ]:
#!/usr/bin/env python

'''
Camshift tracker
================

This is a demo that shows mean-shift based tracking
You select a color objects such as your face and it tracks it.
This reads from video camera (0 by default, or the camera number the user enters)

[1] http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.14.7673

Usage:
------
    camshift.py [<video source>]

    To initialize tracking, select the object with mouse

Keys:
-----
    ESC   - exit
    b     - toggle back-projected probability visualization
'''

# Python 2/3 compatibility
from __future__ import print_function
import sys
PY3 = sys.version_info[0] == 3

if PY3:
    xrange = range

import numpy as np
import cv2 as cv

# local module
import video
from video import presets

# Define a class for the application
class App(object):
    def __init__(self, video_src):
        # Create a video capture object
        self.cam = video.create_capture(video_src, presets['cube'])
        # Read the first frame from the video capture object
        _ret, self.frame = self.cam.read()
        # Create a named window for the application
        cv.namedWindow('camshift')
        # Set a mouse callback function for the named window
        cv.setMouseCallback('camshift', self.onmouse)

        # Initialize variables for the selection, drag start, back projection, and track window
        self.selection = None
        self.drag_start = None
        self.show_backproj = False
        self.track_window = None

    # Define a function for the mouse callback
    def onmouse(self, event, x, y, flags, param):
        if event == cv.EVENT_LBUTTONDOWN:
            self.drag_start = (x, y)
            self.track_window = None
        if self.drag_start:
            xmin = min(x, self.drag_start[0])
            ymin = min(y, self.drag_start[1])
            xmax = max(x, self.drag_start[0])
            ymax = max(y, self.drag_start[1])
            self.selection = (xmin, ymin, xmax, ymax)
        if event == cv.EVENT_LBUTTONUP:
            self.drag_start = None
            self.track_window = (xmin, ymin, xmax - xmin, ymax - ymin)

    # Define a function to show the histogram
    def show_hist(self):
        bin_count = self.hist.shape[0]
        bin_w = 24
        img = np.zeros((256, bin_count*bin_w, 3), np.uint8)
        for i in xrange(bin_count):
            h = int(self.hist[i])
            cv.rectangle(img, (i*bin_w+2, 255), ((i+1)*bin_w-2, 255-h), (int(180.0*i/bin_count), 255, 255), -1)
        img = cv.cvtColor(img, cv.COLOR_HSV2BGR)
        cv.imshow('hist', img)

    # Define a function to run the application
    def run(self):
        while True:
            # Read the next frame from the video capture object
            _ret, self.frame = self.cam.read()
            # Create a copy of the frame for visualization
            vis = self.frame.copy()
            # Convert the frame to the HSV color space
            hsv = cv.cvtColor(self.frame, cv.COLOR_BGR2HSV)
            # Create a binary mask for the selected object
            mask = cv.inRange(hsv, np.array((0., 60., 32.)), np.array((180., 255., 255.)))

            # If an object is selected, calculate the histogram and show it
            if self.selection:
                x0, y0, x1, y1 = self.selection
                hsv_roi = hsv[y0:y1, x0:x1]
                mask_roi = mask[y0:y1, x0:x1]
                hist = cv.calcHist( [hsv_roi], [0], mask_roi, [16], [0, 180] )
                cv.normalize(hist, hist, 0, 255, cv.NORM_MINMAX)
                self.hist = hist.reshape(-1)
                self.show_hist()

                vis_roi = vis[y0:y1, x0:x1]
                cv.bitwise_not(vis_roi, vis_roi)
                vis[mask == 0] = 0

            # If a track window is defined, apply CamShift to the back projection of the histogram
            if self.track_window and self.track_window[2] > 0 and self.track_window[3] > 0:
                self.selection = None
                prob = cv.calcBackProject([hsv], [0], self.hist, [0, 180], 1)
                prob &= mask
                term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1 )
                track_box, self.track_window = cv.CamShift(prob, self.track_window, term_crit)

                # If back projection visualization is enabled, show the back projection
                if self.show_backproj:
                    vis[:] = prob[...,np.newaxis]
                try:
                    # Draw an ellipse around the tracked object
                    cv.ellipse(vis, track_box, (0, 0, 255), 2)
                except:
                    print(track_box)

            # Show the visualization in the named window
            cv.imshow('camshift', vis)

            # Wait for a key press and check for exit or back projection toggle
            ch = cv.waitKey(5)
            if ch == 27:
                break
            if ch == ord('b'):
                self.show_backproj = not self.show_backproj
        # Destroy all windows when the application is finished
        cv.destroyAllWindows()


if __name__ == '__main__':
    print(__doc__)
    import sys
    try:
        video_src = sys.argv[1]
    except:
        video_src = 0
    # Create an instance of the application and run it
    App(video_src).run()